# ML Tutorial (Part 1)

Link to tutorial [here](https://developers.redhat.com/articles/2021/05/21/introduction-machine-learning-jupyter-notebooks#using_a_jupyter_notebook_for_machine_learning)

## Step 1: Explor raw data
* Import required Python libraries
* Convert the raw data file (raw-data.csv) to a DataFrame

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('raw-data.csv')
df['time'] = pd.to_datatime(df['ts'], unit='ms')
df.set_index('time', inplace=True)
df.drop(column=['ts'], inplace=True)

ModuleNotFoundError: No module named 'pandas'